In [1]:
import pandas as pd
import streamlit as st
from src.loaders import get_worksheet, get_worksheet_location
import streamlit as st
import pandas as pd
from src.models import Venue, Cheki, Character, Person, Act

2023-05-27 22:33:39.855 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2023-05-27 22:33:39.859 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2023-05-27 22:33:39.861 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager


In [2]:
sheet_url = st.secrets["private_gsheets_url"]

In [3]:
cheki_df = get_worksheet(sheet_url, 0)
person_df = get_worksheet(sheet_url, 1)
venue_df = get_worksheet_location(sheet_url, 3)

2023-05-27 22:33:40.209 
  command:

    streamlit run /home/wotashu/.cache/pypoetry/virtualenvs/chekilytics-307NsMUd-py3.11/lib/python3.11/site-packages/ipykernel_launcher.py [ARGUMENTS]
2023-05-27 22:33:40.211 No runtime found, using MemoryCacheStorageManager
2023-05-27 22:33:42.464 No runtime found, using MemoryCacheStorageManager


In [4]:
venue_df.head()

,location,neighborhod,city,prefecture,country,full_address,latitude,longitude
1,StringFilter,StringFilter,StringFilter,CSVFilter,CategoryFilter,MapsTooltip - Hidden - StringFilter,MapsLat - Hidden - NoFilter,MapsLong - Hidden - NoFilter
2,Nikkei National Museum & Cultural Centre,Middlegate,Burnaby,BC,Canada,Nikkei National Museum & Cultural Centre Burna...,49.2177915,-122.9645849
3,IdolCon,Twitch,Online,Internet,USA,IdolCon Online Internet USA,37.09024,-95.712891
4,Best Western Plus Silverdale Beach Hotel,Silverdale,Silverdale,Washington,USA,Best Western Plus Silverdale Beach Hotel Silve...,47.6491477,-122.6910949
5,Vancouver Japanese Language School,Japantown,Vancouver,BC,Canada,Vancouver Japanese Language School Vancouver B...,49.2841136,-123.0937815


In [5]:
venues = []
for row in venue_df.itertuples():
    venue_id = row.location.replace(" ", "_").lower()
    venue = Venue(
        venue_id=venue_id,
        title=row.location,
        latitude=row.latitude,
        longitude=row.longitude,
        country=row.country,
        subdivision=row.prefecture,
        municipality=row.city,
        neighborhood=row.neighborhod,
        full_address=row.full_address
    )
    venues.append(venue)

In [6]:
venues_updated = pd.DataFrame([dict(a) for a in venues])

In [7]:
venues_updated.to_parquet("venues.parquet")

In [8]:
venues_updated.to_json("venues.json", orient='records', force_ascii=False, indent=4)

In [9]:
venues_updated.head()

,venue_id,title,latitude,longitude,full_address,postal_code,country,subdivision,municipality,neighborhood
0,stringfilter,StringFilter,MapsLat - Hidden - NoFilter,MapsLong - Hidden - NoFilter,MapsTooltip - Hidden - StringFilter,None,CategoryFilter,CSVFilter,StringFilter,StringFilter
1,nikkei_national_museum_&_cultural_centre,Nikkei National Museum & Cultural Centre,49.2177915,-122.9645849,Nikkei National Museum & Cultural Centre Burna...,None,Canada,BC,Burnaby,Middlegate
2,idolcon,IdolCon,37.09024,-95.712891,IdolCon Online Internet USA,None,USA,Internet,Online,Twitch
3,best_western_plus_silverdale_beach_hotel,Best Western Plus Silverdale Beach Hotel,47.6491477,-122.6910949,Best Western Plus Silverdale Beach Hotel Silve...,None,USA,Washington,Silverdale,Silverdale
4,vancouver_japanese_language_school,Vancouver Japanese Language School,49.2841136,-123.0937815,Vancouver Japanese Language School Vancouver B...,None,Canada,BC,Vancouver,Japantown


In [10]:
person_df.head()

,entity,name1,group1,name2,group2,name3,group3,name4,group4,name5,group5
1,勝月蘭丸@1682,勝月蘭丸,1682,,,,,,,,
2,承和 薫@1682,承和 薫,1682,,,,,,,,
3,月見夜零@1682,月見夜零,1682,,,,,,,,
4,神山おだし@1682,神山おだし,1682,,,,,,,,
5,天野ひろこ@.BPM,天野ひろこ,.BPM,,,,,,,,


In [11]:
persons = {}
acts = {}

for _, row in person_df.iterrows():
    entity = row.entity
    person_id = entity.replace(" ", "_").lower()
    entity_name = entity.split("@")[0]

    characters = []
    for i in range(1, 6):
        name_col = f"name{i}"
        group_col = f"group{i}"

        act = None
        character = None
        if row[group_col]:
            act_id = row[group_col]
            act = Act(title=row[group_col], act_id=act_id)
            acts[act_id] = act

        if row[name_col]:
            character = Character(
                title=row[name_col],
                name=row[name_col],
                act=act_id,
            )
            characters.append(character)
    person = Person(
        person_id=person_id,
        title=entity,
        name=entity_name,
        characters=characters
        )
    persons[person_id] = person

In [12]:
persons_update = pd.DataFrame(data=[dict(value) for value in persons.values()])

In [13]:
persons_update.to_json("persons.json", force_ascii=False, orient="records", indent=4)

In [14]:
persons_update

,person_id,title,name,date_of_birth,characters
0,勝月蘭丸@1682,勝月蘭丸@1682,勝月蘭丸,None,[title='勝月蘭丸' name='勝月蘭丸' act='1682' member_co...
1,承和_薫@1682,承和 薫@1682,承和 薫,None,[title='承和 薫' name='承和 薫' act='1682' member_co...
2,月見夜零@1682,月見夜零@1682,月見夜零,None,[title='月見夜零' name='月見夜零' act='1682' member_co...
3,神山おだし@1682,神山おだし@1682,神山おだし,None,[title='神山おだし' name='神山おだし' act='1682' member_...
4,天野ひろこ@.bpm,天野ひろこ@.BPM,天野ひろこ,None,[title='天野ひろこ' name='天野ひろこ' act='.BPM' member_...
...,...,...,...,...,...
491,山川_さくら@おちゃメンタル☆パーティー,山川 さくら@おちゃメンタル☆パーティー,山川 さくら,None,[title='山川 さくら' name='山川 さくら' act='おちゃメンタル☆パーテ...
492,釈迦力まひな@バブルバビデガム,釈迦力まひな@バブルバビデガム,釈迦力まひな,None,[title='釈迦力まひな' name='釈迦力まひな' act='バブルバビデガム' m...
493,pollie@zero_project,POLLIE@Zero Project,POLLIE,None,[title='POLLIE' name='POLLIE' act='Zero Projec...
494,花紀_舞乃@curtia,花紀 舞乃@CURTiA,花紀 舞乃,None,[title='花紀 舞乃' name='花紀 舞乃' act='CURTiA' membe...


In [19]:
acts_updated = pd.DataFrame(data=[dict(value) for value in acts.values()])

In [20]:
acts_updated.to_json("acts.json", orient="records", force_ascii=False, indent=4)